In [ ]:
import spotipy
import spotipy.util as util
import webbrowser
import json
import urllib.request
from nltk.corpus import stopwords
from nltk import pos_tag
import requests
import collections
import nltk

In [ ]:
# key_file is a variable representing my key txt file
# the 'r' file 
with open("news_key.txt", 'r') as key_file:
    news_key = key_file.read()
# close the file
key_file.close()
print(news_key)

In [ ]:
country = "us"
category = "technology"
url = f"https://newsapi.org/v2/top-headlines?country={country}&category={category}&apiKey={news_key}"
print(url)

In [ ]:
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)

In [ ]:
news_articles = json.loads(response.read())

In [ ]:
news_articles

In [ ]:
# make a list of all the article titles
article_titles = [article['title'] for article in news_articles['articles']]
article_titles

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
tagged_words = pos_tag(article_titles[0].split())
proper_nouns = [word for word,tag in tagged_words if tag == 'NNP']
proper_nouns

In [ ]:
keywords = []
my_lista = ['Apple', 'Vision', 'Pro']
my_listb = ['Sony', 'Playstation']
keywords.extend(my_lista)
keywords.extend(my_listb)
keywords

In [ ]:
keywords = []
for article_title in article_titles:
    # tagged_words will return a list of tuples
    tagged_words = pos_tag(article_title.split())
    # collect all the proper nouns
    keywords.extend([word for word,tag in tagged_words if tag == 'NNP'])

keywords

In [ ]:
word_counter = collections.Counter(keywords)
word_counter

In [ ]:
cred = "spotify_keys.json"
with open(cred, "r") as key_file:
    api_tokens = json.load(key_file)

key_file.close()

In [ ]:
client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username']

In [ ]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirectURI)

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
songs_for_playlist =[]
for keyword in word_counter.keys():
    if word_counter[keyword] > 1:
        returned_song = sp.search(q="artist:"+keyword, type="track", limit=1)
        # make sure something gets returned
        if len(returned_song['tracks']['items']) > 0:
            songs_for_playlist.append(returned_song['tracks']['items'][0]['uri'])

In [ ]:
my_playlist = sp.user_playlist_create(user=username, name="Today's News", public=True, description="a cheeky little playlist for today")
playlist_with_songs = sp.user_playlist_add_tracks(username, my_playlist['id'], songs_for_playlist)

In [ ]:
my_playlist

In [ ]:
webbrowser.open(my_playlist['external_urls']['spotify'])